<a href="https://colab.research.google.com/github/ashinkrishnan/Fake-news-classifier-using-LSTM/blob/main/Fake_news_classifer_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [5]:
df= pd.read_csv('/content/drive/MyDrive/fake_news/train.csv')

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df=df.dropna()
df.shape

(18285, 5)

In [9]:
X=df.drop('label',axis=1)

In [10]:
y=df['label']

In [11]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [19]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

import nltk
import re
from nltk.corpus import stopwords

In [15]:
messages=X.copy()
messages.reset_index(inplace=True)

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus=[]

for i in range(0,len(messages)):
  print(i,end='\r')
  rw = re.sub('^[a-zA-Z]',' ',messages['title'][i])
  rw = rw.lower()
  rw = rw.split()

  rw = [ps.stem(word) for word in rw if not word in stopwords.words('english')]
  rw = ' '.join(rw)
  corpus.append(rw)

In [23]:
corpus[:5]

['ous dem aide: didn’t even see comey’ letter jason chaffetz tweet',
 'lynn: hillari clinton, big woman campu - breitbart',
 'hy truth might get fire',
 '15 civilian kill singl us airstrik identifi',
 'ranian woman jail fiction unpublish stori woman stone death adulteri']

In [27]:
voc_size=5000
onehot_encoded = [one_hot(word,voc_size) for word in corpus]
onehot_encoded[:5]

[[901, 830, 1009, 3315, 870, 2434, 2482, 4177, 3156, 1524, 4154],
 [2874, 3148, 736, 2459, 1895, 1655, 4641],
 [3685, 816, 391, 2869, 4582],
 [2294, 4898, 1131, 690, 3506, 3174, 4443],
 [746, 1895, 3525, 4153, 2646, 768, 1895, 4205, 4552, 1884]]

In [29]:
#Embedding Representation
sent_length = 20
embedded_docs = pad_sequences(onehot_encoded,padding='post',maxlen=sent_length)
print(embedded_docs)

[[ 901  830 1009 ...    0    0    0]
 [2874 3148  736 ...    0    0    0]
 [3685  816  391 ...    0    0    0]
 ...
 [3341 3509 4582 ...    0    0    0]
 [4654 2979 3743 ...    0    0    0]
 [4976 2744 3990 ...    0    0    0]]


In [31]:
## Create Model
embedding_vector_feature = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_feature,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [34]:
### Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 37ms/step - loss: 0.3256 - accuracy: 0.8376 - val_loss: 0.2043 - val_accuracy: 0.9153
Epoch 2/10
192/192 [==============================] - 6s 34ms/step - loss: 0.1344 - accuracy: 0.9491 - val_loss: 0.2456 - val_accuracy: 0.9132
Epoch 3/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0831 - accuracy: 0.9699 - val_loss: 0.2720 - val_accuracy: 0.9102
Epoch 4/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0546 - accuracy: 0.9818 - val_loss: 0.3603 - val_accuracy: 0.9072
Epoch 5/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0336 - accuracy: 0.9892 - val_loss: 0.3830 - val_accuracy: 0.9052
Epoch 6/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0242 - accuracy: 0.9919 - val_loss: 0.3826 - val_accuracy: 0.9032
Epoch 7/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0217 - accuracy: 0.9930 - val_loss: 0.4849 - val_accuracy: 0.9029

In [35]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model_01=Sequential()
model_01.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model_01.add(Dropout(0.3))
model_01.add(LSTM(100))
model_01.add(Dropout(0.3))
model_01.add(Dense(1,activation='sigmoid'))
model_01.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])



In [36]:
### Model Training
model_01.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 37ms/step - loss: 0.3325 - accuracy: 0.8331 - val_loss: 0.2047 - val_accuracy: 0.9160
Epoch 2/10
192/192 [==============================] - 7s 35ms/step - loss: 0.1455 - accuracy: 0.9421 - val_loss: 0.2122 - val_accuracy: 0.9160
Epoch 3/10
192/192 [==============================] - 7s 35ms/step - loss: 0.1013 - accuracy: 0.9624 - val_loss: 0.2332 - val_accuracy: 0.9042
Epoch 4/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0664 - accuracy: 0.9775 - val_loss: 0.2907 - val_accuracy: 0.9085
Epoch 5/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0476 - accuracy: 0.9842 - val_loss: 0.3025 - val_accuracy: 0.9107
Epoch 6/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0337 - accuracy: 0.9891 - val_loss: 0.3850 - val_accuracy: 0.9056
Epoch 7/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0290 - accuracy: 0.9900 - val_loss: 0.4062 - val_accuracy: 0.9049

In [40]:
#preformance metrics
y_pred = model_01.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [41]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3108,  311],
       [ 261, 2355]])

In [42]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9052195526097763